In [1]:
from  __future__ import absolute_import
from __future__ import print_function
from keras.layers import *
from keras.models import Model, load_model
from keras.preprocessing.image import load_img, img_to_array
import tensorflow as tf
from keras import backend as K
import tensorflow_hub as hub
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import logging
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow_hub as hub
from PIL import Image
import pickle
import shutil
import matplotlib.pyplot as plt
import os
import random
import tensorflow as tf
from pathlib import Path
from keras import applications
from keras import layers
from keras import losses
from keras import optimizers
from keras import metrics
from keras import Model
from keras.applications import resnet
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
import tensorflow_addons as tfa
from keras.regularizers import l2
from keras.activations import *
from DistanceLayer import DistanceLayer
from SiameseModel import SiameseModel
import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)

print(tf.__version__)


batch_size = 19




2.8.0


In [2]:


train_triplets = np.loadtxt('./train_triplets_split.txt', dtype=str,delimiter = ' ')
val_triplets = np.loadtxt('./val_triplets_split.txt', dtype=str,delimiter = ' ')



In [3]:
for file in ["xl","resnet","clip","vit"]:

    features = {}
    # file = "clip"
    with open('features_' + file + '.pickle', 'rb') as handle:
        features = pickle.load(handle)

    feature_dim = features["02461"].shape[0]

    split_id = 1
    n = np.array(train_triplets).shape[0]//3
    for split in [train_triplets[i:i + n] for i in range(0, len(train_triplets)-1, n)]:

        print("file: ", file, "split: ", split_id)
        embedding = tf.keras.Sequential([
            Input(shape=(feature_dim,)),
            # BatchNormalization(),
            Dense(8192, activation='relu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.001)),
            # Dropout(0.6),
            BatchNormalization(),
            Dense(8192, activation='relu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.001)),
            # Dropout(0.6),
            BatchNormalization(),
            Lambda(lambda  x: K.l2_normalize(x,axis=1))
        ])

        anchor_input = layers.Input(name="anchor", shape=(feature_dim))
        positive_input = layers.Input(name="positive", shape=(feature_dim))
        negative_input = layers.Input(name="negative", shape=(feature_dim))

        distances = DistanceLayer()(
            embedding(anchor_input),
            embedding(positive_input),
            embedding(negative_input),
        )

        model = Model(
            inputs=[anchor_input, positive_input, negative_input], outputs=distances
        )

        # model.summary()





        #convert the features dictionary to a tensorflow hash table
        keys = tf.constant([key for key in features.keys()])
        values = tf.constant([val for val in features.values()])

        table = tf.lookup.experimental.DenseHashTable(
            value_dtype=tf.float32,
            key_dtype=tf.string,
            empty_key="empty_key",
            deleted_key="deleted_key",
            default_value=[-1]*feature_dim,
            )

        table.insert(keys, values)







        train_dataset = tf.data.Dataset.from_tensor_slices(split)
        val_dataset = tf.data.Dataset.from_tensor_slices(val_triplets)
        count = np.array(split).shape[0]
        # print(count)
        # print(np.array(val_triplets).shape[0])


        @tf.function
        def load_image(inputs):
            filenames = inputs
            anchor = table[filenames[0]]
            positive = table[filenames[1]]
            negative = table[filenames[2]]
            output = (anchor,positive,negative)
            
            return output

        train_dataset = train_dataset.map(load_image).cache().shuffle(buffer_size=1000).batch(batch_size).prefetch(20)
        val_dataset = val_dataset.map(load_image).cache().batch(100).prefetch(20)

        model = SiameseModel(model,margin=0.5,scale=64)
        model.compile(optimizer=tf.keras.optimizers.Adam(0.000001))


        train_steps_per_epoch = int(count/batch_size)
        # checkpoint_filepath = './checkpoints/checkpoint-{epoch}/'
        #save checkpoint after every epoch
        # callback = ModelCheckpoint(
        #     filepath=checkpoint_filepath,
        #     # save_weights_only=True,
        # )
        callback = EarlyStopping(monitor='val_accuracy', patience=4,restore_best_weights=True)


        model.fit(train_dataset, epochs=100,steps_per_epoch=train_steps_per_epoch,batch_size=batch_size,callbacks=[callback],validation_data=val_dataset)



        #predict distances and save them to a file

        dataset = np.loadtxt('./val_triplets_split_shuffled.txt', dtype=str,delimiter = ' ')
        # print(dataset.shape)
        dataset = tf.data.Dataset.from_tensor_slices(dataset)
        dataset = dataset.map(load_image).batch(100)
        left = []
        right = []
        for triplet in iter(dataset):
            anchor = triplet[0]
            positive = triplet[1]
            negative = triplet[2]
            distances = model.predict([anchor,positive,negative])
            left_distances = distances[0]
            right_distances = distances[1]
            s = np.array(left_distances).shape[0]
            # print(s)
            for i in range(s):
                left.append(distances[0][i])
                right.append(distances[1][i])



        df = pd.read_csv("val_distances.csv",index_col=0)
        df['anchor_left_distance_'+file+'_'+str(split_id)] = left
        df['anchor_right_distance_'+file+'_'+str(split_id)] = right
        df.to_csv('val_distances.csv')

        split_id += 1


file:  xl split:  1
128/128 [==============================] - 3s 12ms/step - loss: 0.4607 - val_accuracy: 0.6379
file:  xl split:  2
128/128 [==============================] - 2s 12ms/step - loss: 0.4613 - val_accuracy: 0.6330
file:  xl split:  3
128/128 [==============================] - 2s 11ms/step - loss: 0.4643 - val_accuracy: 0.6397
file:  resnet split:  1
128/128 [==============================] - 2s 12ms/step - loss: 0.4609 - val_accuracy: 0.6421
file:  resnet split:  2
128/128 [==============================] - 2s 12ms/step - loss: 0.4607 - val_accuracy: 0.6650
file:  resnet split:  3
128/128 [==============================] - 2s 12ms/step - loss: 0.4627 - val_accuracy: 0.6547
file:  clip split:  1
128/128 [==============================] - 2s 11ms/step - loss: 0.4624 - val_accuracy: 0.6541
file:  clip split:  2
128/128 [==============================] - 2s 11ms/step - loss: 0.4684 - val_accuracy: 0.6520
file:  clip split:  3
128/128 [==============================] - 2s 11ms

# Model

a lot of the code comes from https://keras.io/examples/vision/siamese_network/
and https://github.com/akarshzingade/image-similarity-deep-ranking/blob/master/deepRanking.py

# this code was used to initialize the distances file

In [4]:
# #predict distances and save them to a file
# set_file = "val"
# dataset = np.loadtxt('./'+set_file+'_triplets_split_shuffled.txt', dtype=str,delimiter = ' ')
# print(dataset.shape)
# dataset = tf.data.Dataset.from_tensor_slices(dataset)
# dataset = dataset.map(load_image).batch(100)
# data = []
# for triplet in iter(dataset):
#     anchor = triplet[0]
#     positive = triplet[1]
#     negative = triplet[2]
#     distances = model.predict([anchor,positive,negative])
#     s = np.array(distances).shape[1]
#     for i in range(s):
#         data.append([distances[0][i],distances[1][i]])



# df = pd.DataFrame(data=data,columns=['anchor_left_distance_'+file,'anchor_right_distance_'+file])
# df.to_csv(set_file+'_distances.csv')




# append other distances

In [5]:
# #predict distances and save them to a file
# set_file = "val"
# dataset = np.loadtxt('./'+set_file+'_triplets_split_shuffled.txt', dtype=str,delimiter = ' ')
# print(dataset.shape)
# dataset = tf.data.Dataset.from_tensor_slices(dataset)
# dataset = dataset.map(load_image).batch(100)
# left = []
# right = []
# for triplet in iter(dataset):
#     anchor = triplet[0]
#     positive = triplet[1]
#     negative = triplet[2]
#     distances = model.predict([anchor,positive,negative])
#     left_distances = distances[0]
#     right_distances = distances[1]
#     s = np.array(left_distances).shape[0]
#     # print(s)
#     for i in range(s):
#         left.append(distances[0][i])
#         right.append(distances[1][i])



# df = pd.read_csv(set_file+"_distances.csv",index_col=0)
# df['anchor_left_distance_'+file] = left
# df['anchor_right_distance_'+file] = right
# df.to_csv(set_file+'_distances.csv')



# using clip directly (sucks)

In [6]:
# #predict distances and save them to a file
# set_file = "val"
# dataset = np.loadtxt('./'+set_file+'_triplets_split_shuffled.txt', dtype=str,delimiter = ' ')
# print(dataset.shape)
# left = []
# right = []
# for triplet in iter(dataset):
#     anchor = triplet[0]
#     positive = triplet[1]
#     negative = triplet[2]
#     # print(np.sum((features[anchor])**2))
#     left.append(np.sum((features[anchor]-features[positive])**2))
#     right.append(np.sum((features[anchor]-features[negative])**2))


# df = pd.read_csv(set_file+"_distances.csv",index_col=0)
# df['anchor_left_distance_clip2'] = left
# df['anchor_right_distance_clip2'] = right
# df.to_csv(set_file+'_distances.csv')

